In [1]:
import os
os.chdir('../..')

In [2]:
from platform import python_version
print(python_version())

3.7.9


In [3]:
import numpy as np
import pandas as pd

import tensorflow as tf

import matplotlib.pyplot as plt

from modules.datgan_preLSTM.model import DATGAN
#from modules.datgan_no_att.model import DATGAN
import networkx as nx

# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [4]:
dataset = 'LPMC'

In [5]:
df = pd.read_csv('../data/{}/data.csv'.format(dataset), index_col=False)

In [6]:
df.head()

,travel_mode,purpose,fueltype,faretype,bus_scale,survey_year,travel_year,travel_month,travel_date,day_of_week,...,dur_pt_access,dur_pt_rail,dur_pt_bus,dur_pt_int,pt_n_interchanges,dur_driving,cost_transit,cost_driving_fuel,cost_driving_con_charge,driving_traffic_percent
0,drive,HBO,Petrol_Car,full,1.0,1,2012,4,1,7,...,0.134444,0.0,0.016667,0.0,0,0.052222,1.5,0.14,0.0,0.111702
1,drive,HBO,Petrol_Car,full,1.0,1,2012,4,1,7,...,0.109444,0.0,0.055556,0.0,0,0.059444,1.5,0.15,0.0,0.112150
2,drive,HBO,Petrol_Car,full,1.0,1,2012,4,1,7,...,0.203056,0.0,0.210278,0.0,0,0.236667,1.5,0.79,0.0,0.203052
3,drive,HBO,Petrol_Car,full,1.0,1,2012,4,1,7,...,0.205556,0.0,0.258611,0.0,0,0.233333,1.5,0.78,0.0,0.160714
4,drive,HBO,Petrol_Car,dis,1.0,1,2012,4,1,7,...,0.203056,0.0,0.189444,0.0,0,0.229167,1.5,0.78,0.0,0.130909


In [7]:
if dataset is 'Chicago':
    continuous_columns = ["distance", "age", "departure_time"]
elif dataset is 'LPMC':
    continuous_columns = ['start_time_linear', 'age', 'distance', 'dur_walking', 'dur_cycling', 'dur_pt_access', 'dur_pt_rail', 'dur_pt_bus', 'dur_pt_int', 'dur_driving', 'cost_transit', 'cost_driving_fuel', 'driving_traffic_percent']

In [8]:
# personalised graph
graph = nx.DiGraph()

if dataset is 'Chicago':
    graph.add_edges_from([
        ("age", "license"),
        ("age", "education_level"),
        ("gender", "work_status"),
        ("education_level", "work_status"),
        ("work_status", "hh_income"),
        ("education_level", "hh_income"),
        ("hh_income", "hh_descr"),
        ("hh_income", "hh_size"),
        ("hh_size", "hh_vehicles"),
        ("hh_size", "hh_bikes"),
        ("work_status", "trip_purpose"),
        ("trip_purpose", "departure_time"),
        ("trip_purpose", "distance"),
        ("travel_dow", "choice"),
        ("distance", "choice"),
        ("departure_time", "choice"),
        ("hh_vehicles", "choice"),
        ("hh_bikes", "choice"),
        ("license", "choice"),
        # Non necessary links
        ("education_level", "hh_size"),
        ("work_status", "hh_descr"),
        ("work_status", "hh_size"),
        ("hh_income", "hh_bikes"),
        ("hh_income", "hh_vehicles"),
        ("trip_purpose", "choice")
    ])
elif dataset is 'LPMC':
    graph.add_edges_from([
        ("travel_year", "survey_year"),
        ("travel_date", "day_of_week"),
        ("day_of_week", "purpose"),
        ("purpose", "start_time_linear"),
        ("purpose", "cost_driving_con_charge"),
        ("purpose", "distance"),
        ("day_of_week", "driving_traffic_percent"),
        ("day_of_week", "cost_driving_con_charge"),
        ("start_time_linear", "driving_traffic_percent"),
        ("start_time_linear", "cost_driving_con_charge"),
        ("driving_traffic_percent", "cost_driving_con_charge"),
        ("female", "driving_license"),
        ("age", "bus_scale"),
        ("age", "car_ownership"),
        ("age", "driving_license"),
        ("age", "faretype"),
        ("driving_license", "car_ownership"),
        ("car_ownership", "fueltype"),
        ("fueltype", "cost_driving_con_charge"),
        ("fueltype", "cost_driving_fuel"),
        ("distance", "cost_driving_fuel"),
        ("distance", "dur_driving"),
        ("distance", "dur_walking"),
        ("distance", "dur_cycling"),
        ("distance", "dur_pt_access"),
        ("distance", "dur_pt_rail"),
        ("distance", "dur_pt_bus"),
        ("distance", "dur_pt_int"),
        ("dur_pt_bus", "cost_transit"),
        ("dur_pt_rail", "cost_transit"),
        ("pt_n_interchanges", "dur_pt_int"),
        ("pt_n_interchanges", "cost_transit"),
        ("faretype", "cost_transit"),
        ("bus_scale", "cost_transit"),
        ("car_ownership", "travel_mode"),
        ("age", "travel_mode"),
        ("cost_driving_con_charge", "travel_mode"),
        ("driving_traffic_percent", "travel_mode"),
        ("female", "travel_mode"),
        ("purpose", "travel_mode"),
        ("cost_transit", "travel_mode"),
        ("cost_driving_fuel", "travel_mode"),
        ("dur_driving", "travel_mode"),
        ("dur_walking", "travel_mode"),
        ("dur_cycling", "travel_mode"),
        ("dur_pt_access", "travel_mode"),
        ("dur_pt_rail", "travel_mode"),
        ("dur_pt_bus", "travel_mode"),
        ("dur_pt_int", "travel_mode")
    ])
    graph.add_node("travel_month")

In [9]:
output_folder = '../output/' + dataset + '/DATGAN/'

In [10]:
datgan = DATGAN(continuous_columns, max_epoch=1000, steps_per_epoch=2000, batch_size=200, output=output_folder, gpu=0)

In [ ]:
datgan.fit(df, graph)




[0922 08:43:07 @input_source.py:222] Setting up the queue 'QueueInput/input_queue' for CPU prefetching ...








Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
-> Creating cell for travel_year (in-edges: 0)
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
[0922 08:43:07 @registry.py:126] gen/LSTM/00/FC input: [200, 100]

Instructions for updating:
Please use `layer.__call__` method instead.
[0922 08:43:07 @registry.py:134] gen/LSTM/00/FC output: [200, 100]
[0922 08:43:07 @registry.py:126] gen/LSTM/00/FC2 input: [200, 100]
[0922 08:43:07 @registry.py:134] gen/LSTM/00/FC2 output: [200, 4]
[0922 08:43:07 @registry.py:126] gen/LSTM/00/FC3 input: [200, 4]
[0922 08:43:07 @registry.py:134] gen/LSTM/00/FC3 output: [200, 100]
-> Creating cell for travel_date (in-edges: 

-> Creating cell for driving_license (in-edges: 2)
[0922 08:43:08 @registry.py:126] gen/LSTM/09/FC_inputs input: [200, 200]
[0922 08:43:08 @registry.py:134] gen/LSTM/09/FC_inputs output: [200, 100]
[0922 08:43:08 @registry.py:126] gen/LSTM/09/FC_attentions input: [200, 200]
[0922 08:43:08 @registry.py:134] gen/LSTM/09/FC_attentions output: [200, 100]
[0922 08:43:08 @registry.py:126] gen/LSTM/09/FC_lstm_state_0 input: [200, 200]
[0922 08:43:08 @registry.py:134] gen/LSTM/09/FC_lstm_state_0 output: [200, 100]
[0922 08:43:08 @registry.py:126] gen/LSTM/09/FC_lstm_state_1 input: [200, 200]
[0922 08:43:08 @registry.py:134] gen/LSTM/09/FC_lstm_state_1 output: [200, 100]

[0922 08:43:08 @registry.py:126] gen/LSTM/10/FC input: [200, 100]
[0922 08:43:08 @registry.py:134] gen/LSTM/10/FC output: [200, 100]
[0922 08:43:08 @registry.py:126] gen/LSTM/10/FC2 input: [200, 100]
[0922 08:43:08 @registry.py:134] gen/LSTM/10/FC2 output: [200, 2]
[0922 08:43:08 @registry.py:126] gen/LSTM/10/FC3 input: [200, 

[0922 08:43:09 @registry.py:134] gen/LSTM/24/FC_lstm_state_0 output: [200, 100]
[0922 08:43:09 @registry.py:126] gen/LSTM/24/FC_lstm_state_1 input: [200, 200]
[0922 08:43:09 @registry.py:134] gen/LSTM/24/FC_lstm_state_1 output: [200, 100]
[0922 08:43:09 @registry.py:126] gen/LSTM/25/FC input: [200, 100]
[0922 08:43:09 @registry.py:134] gen/LSTM/25/FC output: [200, 100]
[0922 08:43:09 @registry.py:126] gen/LSTM/25/FC2 input: [200, 100]
[0922 08:43:09 @registry.py:134] gen/LSTM/25/FC2 output: [200, 1]
[0922 08:43:09 @registry.py:126] gen/LSTM/26/FC input: [200, 100]
[0922 08:43:09 @registry.py:134] gen/LSTM/26/FC output: [200, 100]
[0922 08:43:09 @registry.py:126] gen/LSTM/26/FC2 input: [200, 100]
[0922 08:43:09 @registry.py:134] gen/LSTM/26/FC2 output: [200, 5]
[0922 08:43:09 @registry.py:126] gen/LSTM/26/FC3 input: [200, 5]
[0922 08:43:09 @registry.py:134] gen/LSTM/26/FC3 output: [200, 100]
-> Creating cell for dur_driving (in-edges: 1)
[0922 08:43:09 @registry.py:126] gen/LSTM/27/FC i

[0922 08:43:10 @registry.py:134] gen/LSTM/43/FC output: [200, 100]
[0922 08:43:10 @registry.py:126] gen/LSTM/43/FC2 input: [200, 100]
[0922 08:43:10 @registry.py:134] gen/LSTM/43/FC2 output: [200, 2]
[0922 08:43:10 @registry.py:126] gen/LSTM/43/FC3 input: [200, 2]
[0922 08:43:10 @registry.py:134] gen/LSTM/43/FC3 output: [200, 100]
-> Creating cell for cost_transit (in-edges: 5)
[0922 08:43:10 @registry.py:126] gen/LSTM/44/FC_inputs input: [200, 500]
[0922 08:43:10 @registry.py:134] gen/LSTM/44/FC_inputs output: [200, 100]
[0922 08:43:10 @registry.py:126] gen/LSTM/44/FC_attentions input: [200, 500]
[0922 08:43:10 @registry.py:134] gen/LSTM/44/FC_attentions output: [200, 100]
[0922 08:43:10 @registry.py:126] gen/LSTM/44/FC_lstm_state_0 input: [200, 500]
[0922 08:43:10 @registry.py:134] gen/LSTM/44/FC_lstm_state_0 output: [200, 100]
[0922 08:43:10 @registry.py:126] gen/LSTM/44/FC_lstm_state_1 input: [200, 500]
[0922 08:43:10 @registry.py:134] gen/LSTM/44/FC_lstm_state_1 output: [200, 100]

[0922 08:43:17 @base.py:209] Setup callbacks graph ...






[0922 08:43:18 @summary.py:46] [MovingAverageSummary] 6 operations in collection 'MOVING_SUMMARY_OPS' will be run with session hooks.
[0922 08:43:18 @summary.py:93] Summarizing collection 'summaries' of size 9.

[0922 08:43:18 @graph.py:98] Applying collection UPDATE_OPS of 4 ops.



[0922 08:43:20 @base.py:230] Creating the session ...


Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.



[0922 08:43:24 @base.py:236] Initializing the session ...
[0922 08:43:24 @sessinit.py:114] Restoring checkpoint from ../output/LPMC/DATGAN/model\model-666000 ...
INFO:tensorflow:Restoring parameters from ../output/LPMC/DATGAN/model\model-666000
[0922 08:43:24 @base.py:243] Graph Finalized.


[0922 08:43:25 @concurrency.py:38] Starting EnqueueThread QueueInput/input_queue ...
[0922 08:43:25 @steps.py:124] Start training with global_step=666000
[0922 08:43:26 @monitor.py:347] Found existing JSON i

...

[0922 19:15:19 @monitor.py:467] GAN_loss/discrim/accuracy_fake: 0.895
[0922 19:15:19 @monitor.py:467] GAN_loss/discrim/accuracy_real: 0.175
[0922 19:15:19 @monitor.py:467] GAN_loss/discrim/loss: 0.65397
[0922 19:15:19 @monitor.py:467] GAN_loss/gen/final-g-loss: 1.1791
[0922 19:15:19 @monitor.py:467] GAN_loss/gen/klloss: 0.12045
[0922 19:15:19 @monitor.py:467] GAN_loss/gen/loss: 1.0587
[0922 19:15:19 @monitor.py:467] QueueInput/queue_size: 50
[0922 19:15:19 @base.py:275] Start Epoch 860 ...


100%|#################################################################################|2000/2000[01:10<00:00,28.20it/s]

[0922 19:16:30 @base.py:285] Epoch 860 (global_step 1720000) finished, time:1 minute 10 seconds.
[0922 19:16:30 @saver.py:79] Model saved to ../output/LPMC/DATGAN/model\model-1720000.


[0922 19:16:31 @monitor.py:467] GAN_loss/discrim/accuracy_fake: 0.885
[0922 19:16:31 @monitor.py:467] GAN_loss/discrim/accuracy_real: 0.205
[0922 19:16:31 @monitor.py:467] GAN_loss/discrim/loss: 0.64974
[0922 19:16:31 @monitor.py:467] GAN_loss/gen/final-g-loss: 1.2538
[0922 19:16:31 @monitor.py:467] GAN_loss/gen/klloss: 0.19141
[0922 19:16:31 @monitor.py:467] GAN_loss/gen/loss: 1.0624
[0922 19:16:31 @monitor.py:467] QueueInput/queue_size: 50
[0922 19:16:31 @base.py:275] Start Epoch 861 ...


 28%|#######################2                                                          |567/2000[00:20<00:49,28.68it/s]

In [13]:
datgan.save('trained', force=True)

[0809 15:13:09 @model.py:813] Model saved successfully.
